## Unit 2 Assignment: Building a Mixture of Experts (MoE) Router
Groq API + Python + Dotenv

In [15]:
!pip3 install groq python-dotenv --quiet

import os
import getpass
from dotenv import load_dotenv
from groq import Groq

# Try loading from .env first
load_dotenv()

# If not found, ask user to enter it securely
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Initialize Groq client
client = Groq(api_key=os.environ["GROQ_API_KEY"])
print("Groq Client Initialized Successfully")


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Groq Client Initialized Successfully


In [16]:
models = client.models.list()
for m in models.data:
    print(m.id)

meta-llama/llama-prompt-guard-2-22m
meta-llama/llama-4-maverick-17b-128e-instruct
canopylabs/orpheus-v1-english
meta-llama/llama-prompt-guard-2-86m
meta-llama/llama-guard-4-12b
whisper-large-v3-turbo
groq/compound-mini
llama-3.3-70b-versatile
groq/compound
openai/gpt-oss-20b
openai/gpt-oss-safeguard-20b
canopylabs/orpheus-arabic-saudi
moonshotai/kimi-k2-instruct-0905
allam-2-7b
whisper-large-v3
llama-3.1-8b-instant
openai/gpt-oss-120b
moonshotai/kimi-k2-instruct
meta-llama/llama-4-scout-17b-16e-instruct
qwen/qwen3-32b


In [19]:
# Switched to Llama since Groq migrated away from Mixtral 8x7B
MODEL_NAME = "llama-3.1-8b-instant"

In [20]:
# Define Experts
MODEL_CONFIG = {
    "technical": {
        "system_prompt": """
You are a Senior Software Engineer.
Be precise, technical, and solution-oriented.
Provide code snippets when necessary.
Explain errors clearly and suggest debugging steps.
"""
    },
    "billing": {
        "system_prompt": """
You are a Customer Billing Specialist.
Be empathetic and professional.
Explain refund policies clearly.
Guide the user through next steps politely.
"""
    },
    "general": {
        "system_prompt": """
You are a friendly general customer support assistant.
Answer casually and clearly.
If unsure, politely ask for clarification.
"""
    }
}

In [21]:
# Router Function
def route_prompt(user_input: str) -> str:
    """
    Classifies user input into one of:
    technical, billing, general
    
    Returns ONLY the category name.
    """
    
    routing_prompt = f"""
Classify the following text into one of these categories:
[technical, billing, general]

Return ONLY the category name.

Text:
{user_input}
"""

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0,  # deterministic routing
        messages=[
            {"role": "system", "content": "You are an intent classifier."},
            {"role": "user", "content": routing_prompt}
        ]
    )
    
    category = response.choices[0].message.content.strip().lower()
    
    # safety fallback
    if category not in MODEL_CONFIG:
        return "general"
    
    return category

In [22]:
# Process Request Function
def process_request(user_input: str) -> str:
    """
    Full MoE flow:
    1. Route intent
    2. Select expert
    3. Generate response
    """
    
    category = route_prompt(user_input)
    system_prompt = MODEL_CONFIG[category]["system_prompt"]
    
    print(f"Routed to: {category.upper()} expert\n")
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0.7,  # creative expert
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )
    
    return response.choices[0].message.content

### Testcases

In [25]:
# Technical Expert
print(process_request("My Python script is throwing an IndexError on line 5."))

Routed to: TECHNICAL expert

**Troubleshooting IndexError in Python**

To solve this issue, we need to identify the cause of the `IndexError`. Here are some general steps to help you diagnose the problem:

1. **Check the index**: Verify that the index being used is within the valid range of the list or array. In Python, indices start at 0, so the last valid index is always one less than the length of the list.

2. **Check for empty lists**: Ensure that the list or array being accessed is not empty, as attempting to access an index on an empty list will raise an `IndexError`.

3. **Verify list length**: Double-check the length of the list or array to ensure it's what you expect.

### Example Code Snippet

Let's assume you have the following Python code:
```python
my_list = [1, 2, 3]
print(my_list[5])  # raises IndexError
```
In this example, the `IndexError` is raised because the index `5` is out of range for a list of length `3`.

### Debugging Steps

1. **Inspect the list length**: Us

In [26]:
# Billing Expert
print(process_request("I was charged twice for my subscription this month."))

Routed to: BILLING expert

I'm so sorry to hear that you've been charged twice for your subscription. I'm here to help you resolve this issue as quickly as possible.

First, let me assure you that we take cases like this seriously and will make every effort to rectify the situation. In order to process a refund, I'll need to verify your account details and the charges that were made.

Could you please provide me with your subscription account information, including your name, email address, and the subscription details? This will help me to locate the duplicate charges and initiate the refund process.

Additionally, I want to let you know that our standard refund policy is as follows:

- We will refund any duplicate charges that were made in error.
- Refunds will be processed back to the original payment method within 3-5 business days.
- You will receive an email confirmation once the refund has been processed.

Please rest assured that we value your business and appreciate your patie

In [28]:
# General Expert
print(process_request("Hi there! What services do you offer?"))

Routed to: GENERAL expert

Hi! We're glad you're here. We offer a variety of services that cater to different needs. We have technical support for a wide range of products, including computers, smartphones, and home appliances. We also have assistance with online services, such as email and social media setup. Additionally, we provide general advice on how to use our software and tools.

If you're looking for something specific, feel free to let me know and I'll do my best to help. What brings you to our support today?


### Tool Expert (BONUS QUESTION)

In [30]:
# Update Model Config- Add Tool Expert
MODEL_CONFIG = {
    "technical": {
        "system_prompt": """
You are a Senior Software Engineer.
Be precise and technical.
Provide code fixes and debugging steps.
"""
    },
    "billing": {
        "system_prompt": """
You are a Billing Support Specialist.
Be empathetic and explain refund policies clearly.
Guide users professionally.
"""
    },
    "general": {
        "system_prompt": """
You are a friendly customer support assistant.
Answer clearly and politely.
"""
    },
    "tool": {
        "system_prompt": """
You are a tool router. If the system calls you,
it means a function should be executed instead of answering normally.
"""
    }
}

In [31]:
# Upgrade Router
def route_prompt(user_input: str) -> str:
    routing_prompt = f"""
You are a strict intent classifier.

Classify the user message into ONE of these categories:
technical
billing
general
tool

Rules:
- If the user asks for real-time data (price, weather, stock, crypto), choose: tool
- If asking about code, bugs, errors → technical
- If asking about payment, charges, refunds → billing
- Otherwise → general

Respond with ONLY one word.

Message:
{user_input}
"""

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0,
        messages=[
            {"role": "system", "content": "You are an intent classifier."},
            {"role": "user", "content": routing_prompt}
        ]
    )

    category = response.choices[0].message.content.strip().lower()

    if category not in MODEL_CONFIG:
        return "general"

    return category

In [32]:
# Mock Function
def get_bitcoin_price():
    return {
        "asset": "Bitcoin",
        "price_usd": 52340,
        "source": "Mock Market API"
    }

In [37]:
# Orchestrator
def process_request(user_input: str) -> str:
    category = route_prompt(user_input)

    print(f"Routed to: {category.upper()}")

    # ---- TOOL HANDLING ----
    if category == "tool":
        tool_result = get_bitcoin_price()
        
        return (
            f"Live Market Data\n"
            f"Asset: {tool_result['asset']}\n"
            f"Price (USD): ${tool_result['price_usd']}\n"
            f"Source: {tool_result['source']}"
        )

    # ---- LLM EXPERT HANDLING ----
    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content

### Testcases

In [38]:
#1 Technical
print(process_request("My Python script throws IndexError on line 5"))

Routed to: TECHNICAL
**IndexError Explanation**

An `IndexError` in Python typically occurs when you're trying to access an element at an index that doesn't exist in a list or other sequence.

**Debugging Steps**

To resolve the issue, follow these steps:

1. **Check the index value**: Verify that the index value you're using is within the valid range for the list. You can do this by printing the length of the list before accessing the element.
2. **Verify the list contents**: Ensure that the list isn't empty or contains the expected elements.
3. **Check for indexing errors**: Double-check your indexing logic to ensure you're not trying to access an index that's out of range.

**Example Code**

Assuming the following code:
```python
my_list = [1, 2, 3]
index = 5

try:
    print(my_list[index])
except IndexError as e:
    print(f"IndexError: {e}")
```
**Error Fix**

The `IndexError` occurs because the index `5` is out of range for the list `[1, 2, 3]`, which has a length of 3 ( indices 

In [40]:
#2 Billing
print(process_request("I was charged twice for my subscription"))

Routed to: BILLING
I'm so sorry to hear that you were charged twice for your subscription. I'm here to help you resolve this issue.

First, let me assure you that we take situations like this very seriously and we're committed to making it right. I'd like to guide you through the process of getting a refund.

To confirm, can you please provide me with your subscription details, such as the order number, the date of the charge, and the amount charged? This information will help me to identify the issue and process a refund for you.

Additionally, please let me know if you've already contacted our support team previously regarding this issue. This will help me to quickly locate your case and expedite the refund process.

In the meantime, I want to let you know that our refund policy is as follows:

* If you've been charged twice for your subscription, we'll refund the duplicate charge as soon as possible.
* We'll also credit your account for any other issues or errors that may have occur

In [ ]:
#3 Tool
# LLMs hallucinate real-time data. For factual, time-sensitive queries, tool routing ensures deterministic and accurate responses.
print(process_request("What is the current price of Bitcoin?"))

Routed to: TOOL
Live Market Data
Asset: Bitcoin
Price (USD): $52340
Source: Mock Market API
